In [1]:
import pandas as pd
import wrangle as w
import explore as e
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from functools import reduce

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

In [2]:
df = w.prep_df()
df.head()

KeyError: "['Unnamed: 0.1'] not found in axis"

In [ ]:
# acquire dataframe
df = w.get_maximum_df()
df.head()

In [ ]:
df.columns

In [ ]:
# drop columns from the dataframe that arent relevant in the predictor
df = df.drop(columns=['month'])
df.head()

In [ ]:
df.country.value_counts()

### Clustering

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
# !pip install kmodes
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cluster_data = data
cluster_data = cluster_data.drop(columns={'gname'})
cluster_data

# Elbow curve to find optimal K
cost = []
K = range(1,5)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(data)
    cost.append(kmode.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Building the model with 3 clusters
kmode = KModes(n_clusters=3, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(cluster_data)
clusters

In [ ]:
data.insert(0, "Cluster", clusters, True)
data

#### Rerunning DTC

In [ ]:
# create a dataframe for the gname target variable
gnames = pd.DataFrame(data.gname)

# create nkill df
nkills = pd.DataFrame(data.nkill)

# scale nkill
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(nkills)
nkills = pd.DataFrame(scaler.transform(nkills))

In [ ]:
gnames.value_counts()

In [ ]:
# rename popped off index
nkills = nkills.rename(columns={0:'nkill'})
nkills

In [ ]:
data.shape[0]

In [ ]:
gnames.shape[0]

In [ ]:
nkills.shape[0]

In [ ]:
data.year = data.year.astype('str')

In [ ]:
data.info()

In [ ]:
# encode variables
dummy_df = pd.get_dummies(data[['country',
                                    'provstate',
                                    'city',
                                    'suicide',
                                    'attack_type', 
                                    'targtype',  
                                    'weaptype',
                                    'year', 
                                    'Cluster']], dummy_na=False, drop_first=[True, True])


In [ ]:
dummy_df = dummy_df.reset_index(drop=True)
dummy_df.index

In [ ]:
gnames = gnames.reset_index(drop=True)
gnames.index

In [ ]:
# add target to dataframe
dummy_df = pd.concat([dummy_df, gnames, nkills], axis=1)

In [ ]:
dummy_df.shape

In [ ]:
# drop previous index, date column
dummy_df.head()

In [ ]:
def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.27, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.35, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [ ]:
# split into train, validate, test
train, validate, test = train_validate_test_split(dummy_df, target='gname', seed=123)

# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['gname'])
y_train = train.gname

X_validate = validate.drop(columns=['gname'])
y_validate = validate.gname

X_test = test.drop(columns=['gname'])
y_test = test.gname

In [ ]:
metrics = []
for i in range(1, 15):
    tree = DecisionTreeClassifier(max_depth=i, random_state=123)
    tree = tree.fit(X_train, y_train)
    in_sample_accuracy = tree.score(X_train, y_train)
    
    out_of_sample_accuracy = tree.score(X_validate, y_validate)
    output = {
        "max_depth": i,
        "train_accuracy": in_sample_accuracy,
        "validate_accuracy": out_of_sample_accuracy
    }
    
    metrics.append(output)
    
dt_df = pd.DataFrame(metrics)
dt_df["difference"] = dt_df.train_accuracy - dt_df.validate_accuracy
dt_df

In [ ]:
X_train.shape, X_validate.shape, X_test.shape

In [ ]:
tree = DecisionTreeClassifier(max_depth=4, random_state=123)

tree = tree.fit(X_train, y_train)

tree.score(X_test, y_test)